In [5]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", sep='\t', encoding='utf-8')
test = pd.read_csv("test.csv", sep='\t', encoding='utf-8')
test_label = pd.read_csv("sample_submission.csv", sep=',', encoding='utf-8')
train.head()
test_label['label']

# type(train) : pandas.core.frame.DataFrame
# train.columns : Index(['text', 'label'], dtype='object')
# print( len(train['text'])): 4987
# print(len(test['text'])): 1247

0       1
1       1
2       0
3       0
4       0
       ..
1242    0
1243    0
1244    1
1245    1
1246    1
Name: label, Length: 1247, dtype: int64

In [6]:
# 把 sample_submission.csv 中的 id 與 label切開，再塞入 test

    
test["label"]=test_label['label']

test

,id,text,label
0,2,The 2017 Teen Choice Awards ceremony was held ...,1
1,3,"The concert, part of “The Joshua Tree Tour,” w...",1
2,4,Selena Gomez refuses to talk to her mother abo...,0
3,5,This is worse than a lump of coal in your stoc...,0
4,6,Luann De Lesseps is going to rehab after her a...,0
...,...,...,...
1242,1244,Get the latest from TODAY Sign up for our news...,0
1243,1245,Jaden Smith claims that the Four Seasons Hotel...,0
1244,1246,Overview (3) Mini Bio (1) Faith Hill was bor...,1
1245,1247,CLOSE Aaron Paul dishes on 'The Path' Aaron P...,1


In [7]:
# train["label"].value_counts()
# 0        2972
# 1        2014
# label       1
# Name: label, dtype: int64
# 有個 label夾在0.1裡面，抓出來刪掉row

# a=0
# for i in train["label"]:
#     if i!="label":
#         a+=1
#     if i=="label":
#         break
# a=1615
train = train.drop([1615])

In [8]:
train[1614:]

,text,label
1614,Justin Bieber has had his fair share of accide...,1
1616,Mel B has filed for divorce from her husband o...,0
1617,Recognizing The Potential And Perils Of A 'Lim...,1
1618,How much did you spend on your actual home? We...,1
1619,She's got a name! After Kim Kardashian and Ka...,1
...,...,...
4982,The storybook romance of WWE stars John Cena a...,0
4983,The actor told friends he’s responsible for en...,0
4984,Sarah Hyland is getting real. The Modern Fami...,0
4985,Production has been suspended on the sixth and...,0


In [9]:
train["label"].value_counts()

0    2972
1    2014
Name: label, dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# 引用 NLTK 的 stop words list
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# 將文字轉成向量，同時 去除停頓詞stop words 
vectorizer = CountVectorizer(stop_words=stop_words,max_features=2500)
train_vector = vectorizer.fit_transform(train["text"]).toarray()
test_vector = vectorizer.fit_transform(test["text"]).toarray()

# print(vectorizer.get_feature_names())


In [ ]:
len(train_vector)

4987

In [11]:
train_X = train_vector
train_y = train["label"].values
test_X = test_vector
test_y = test["label"].values

In [12]:
"""
xgboost
"""
import xgboost as xgb

# read in data
dtrain = xgb.DMatrix(train_X , label=train_y)
dtest = xgb.DMatrix(test_X , label=test_y)
# specify parameters via map
param = {'max_depth': 25, 'eta': 1, 'objective': 'reg:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

num_round = 5 # 2:電腦跑10秒、5:電腦跑30秒外加風扇全開，決定跑到5就好，不要虐待電腦。recall從0.1升到0.2已經夠了。
bst = xgb.train(param, dtrain, num_round)

#After training, the model can be saved.
bst.save_model('0001.model')

#A saved model can be loaded as follows:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('0001.model')  # load data

# make prediction
#preds = bst.predict(dtest)

In [13]:
# 預測數據集 
y_pred = bst.predict(dtest)

In [ ]:
import matplotlib
#畫出 XGB的樹
xgb.to_graphviz(bst)

In [ ]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred.round())
recall = recall_score(test_y,y_pred.round())
accuracy = accuracy_score(test_y,y_pred.round())
f1_measure = f1_score(test_y,y_pred.round())
print("XGboost precision=",precision)
print("XGboost recall=",recall)
print("XGboost accuracy=",accuracy)
print("XGboost f1_score=",f1_measure)


XGboost precision= 0.47985347985347987
XGboost recall= 0.21231766612641814
XGboost accuracy= 0.4963913392141139
XGboost f1_score= 0.2943820224719101


In [14]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred.round())
recall = recall_score(test_y,y_pred.round())
accuracy = accuracy_score(test_y,y_pred.round())
f1_measure = f1_score(test_y,y_pred.round())
print("XGboost precision=",precision)
print("XGboost recall=",recall)
print("XGboost accuracy=",accuracy)
print("XGboost f1_score=",f1_measure)

XGboost precision= 0.4792703150912106
XGboost recall= 0.4683954619124797
XGboost accuracy= 0.48516439454691257
XGboost f1_score= 0.4737704918032787


In [15]:
train_y=[]
for i in train["label"]:
    a = int(i)
    train_y.append(a)

In [16]:
train_X = train_vector
train_y = train_y
test_X = test_vector
test_y = test["label"].values

In [17]:
"""
GBDT
"""
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn import ensemble

# model = ensemble.HistGradientBoostingClassifier()
# model.fit(train_X, train_y)
# y_pred_GBDT=model.predict(test_X)
# 跑超過4分鐘，終於沒有error，但是整個chrome跳出"記憶體滿載"的提示後強制關閉

# 跑超過1小時，只好先中止
# from GBDT.gbdt import GradientBoostingBinaryClassifier
# model = GradientBoostingBinaryClassifier(learning_rate=0.1, max_depth=10)
# model.fit(train_X, train_y)
# y_pred_GBDT=model.predict(test_X)

# 為什麼跑5分鐘都跑不出來東西RRR，又不是多大的模型
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(train_X, train_y)


In [18]:
y_pred_GBDT = clf.predict(test_X)

In [19]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred_GBDT.round())
recall = recall_score(test_y,y_pred_GBDT.round())
accuracy = accuracy_score(test_y,y_pred_GBDT.round())
f1_measure = f1_score(test_y,y_pred_GBDT.round())
print("GBDT precision=",precision)
print("GBDT recall=",recall)
print("GBDT accuracy=",accuracy)
print("GBDT f1_score=",f1_measure)

GBDT precision= 0.5158924205378973
GBDT recall= 0.3419773095623987
GBDT accuracy= 0.5156375300721732
GBDT f1_score= 0.41130604288499023


In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)

clf.predict(X_test[:2])

clf.score(X_test, y_test)

In [ ]:
len(X_train)

75

In [ ]:
X_train

array([[-0.65240858,  0.49374178,  1.30184623, ..., -1.30819171,
        -1.04525337, -0.11054066],
       [ 0.35178011, -0.47003288, -0.37914756, ..., -2.38076394,
        -0.11048941, -1.55042935],
       [-1.58249448, -1.42279491, -0.56430103, ...,  1.26661394,
        -1.31771734,  1.61805427],
       ...,
       [-0.96050438, -2.28862004,  1.02943883, ..., -0.79347019,
         1.12859406, -0.27567053],
       [ 0.91017891,  0.78632796,  0.06326199, ...,  0.42234144,
        -0.46359597, -0.01702041],
       [-0.87916063, -1.63880731, -0.30769128, ..., -0.6054158 ,
         1.57886519,  0.73165893]])

In [ ]:
len(y_train)

75

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0])

In [ ]:
y_test

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0])

In [20]:
"""
Light GBM
"""

import lightgbm as lgb 

train_data = lgb.Dataset(train_X , label=train_y)

param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'

num_round = 10
bst = lgb.train(param, train_data, num_round)

# After training, the model can be saved:
bst.save_model('model.txt')

# A saved model can be loaded:
bst = lgb.Booster(model_file='model.txt')  # init model

ypred_LightGBM = bst.predict(test_X, predict_disable_shape_check="true")

[LightGBM] [Info] Number of positive: 2014, number of negative: 2972
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20373
[LightGBM] [Info] Number of data points in the train set: 4986, number of used features: 2482
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403931 -> initscore=-0.389112
[LightGBM] [Info] Start training from score -0.389112


In [21]:
# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision_LightGBM = precision_score(test_y,ypred_LightGBM.round())
recall_LightGBM = recall_score(test_y,ypred_LightGBM.round())
accuracy_LightGBM = accuracy_score(test_y,ypred_LightGBM.round())
f1_measure_LightGBM = f1_score(test_y,ypred_LightGBM.round())
print("Light GBM precision=",precision_LightGBM)
print("Light GBM recall=",recall_LightGBM)
print("Light GBM accuracy=",accuracy_LightGBM)
print("Light GBM f1_score=",f1_measure_LightGBM)

Light GBM precision= 0.5264900662251656
Light GBM recall= 0.2576985413290113
Light GBM accuracy= 0.5180433039294307
Light GBM f1_score= 0.34602829162132753
